In [ ]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Create an artificial cell (polygon) and surrounding cells

In [ ]:
# the position of the considered cell center
center = np.array([0., 0.])

# The vertices of the considered cells
vertices_x = []
vertices_y = []
ratio = 1.4 # the ratio of the long to short axis of an ellipse
for angle in np.linspace(0, 2 * math.pi, 6, endpoint=False) + np.random.uniform(-0.2, 0.2, size=6) + math.pi/6.:
    vertices_x.append(np.cos(angle)*ratio**0.5)
    vertices_y.append(np.sin(angle)/ratio**0.5)
vertices_x = np.asarray(vertices_x)
vertices_y = np.asarray(vertices_y)

# The coordinates of the neighbouring cells
coordinates_x = []
coordinates_y = []
for angle in np.linspace(0, 2 * math.pi, 6, endpoint=False) + np.random.uniform(-0.2, 0.2, size=6):
    coordinates_x.append(2*np.cos(angle)*ratio**0.5)
    coordinates_y.append(2*np.sin(angle)/ratio**0.5)

angles = np.arctan2(coordinates_y - center[1], coordinates_x - center[0])

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect(1)
ax.scatter(center[0], center[1], color='black', label='center HC')
ax.scatter(vertices_x, vertices_y, color='grey', label='vertices')
ax.quiver(
    vertices_x,
    vertices_y, 
    np.roll(vertices_x, 1) - vertices_x,
    np.roll(vertices_y, 1) - vertices_y,
    angles='xy', scale_units='xy', scale=1,
    headwidth=0, headlength=0,headaxislength=0,
    color='gray',
    label='junctions'
)

ax.scatter(coordinates_x, coordinates_y, color='red', label='HCs')
ax.quiver(
    coordinates_x,
    coordinates_y, 
    center[0] - coordinates_x,
    center[1] - coordinates_y,
    angles='xy', scale_units='xy', scale=1,
    headwidth=0, headlength=0,headaxislength=0,
    color='red',
    # label='junctions'
)


ax.legend(bbox_to_anchor=(1.3, 1))
ax.set_title("Arrangement of HCs and vertices")

In [ ]:
def hexatic_order_parameter(angles):
    """
    Calculate the uncorrected hexatic order parameter.

    Args:
        angles: The angles (rad) contributing to hexatic order
    
    """
    return np.abs((np.exp(6j * angles)).sum()) / angles.shape[0]

In [ ]:
# calculate the uncorrected hexatic parameter for the arrangement of HCs
# around the center cell
print(hexatic_order_parameter(angles))

# Calculate the elongation of the center cell

In [ ]:
    def elongation(vertices_x, vertices_y, center):
        """The elongation of a polygon defined by the set of x and y coordinates.
        
        The coordinates will be sorted anti-clockwise.

        Returns:
            - Elongation tensor Q
            - Area of polygon
            - Angle of rotation
            - absolute of Q tensor
            - the ratio of long to short axis of an ellipse fitting this polygon
        """

        def decompose(mat: np.array):
            """Decomposition of a matrix into trace, symmetric and asymmetric parts
            """
            tr = np.trace(mat)
            sym = 0.5 * (mat + np.transpose(mat)) - tr / 2. * np.eye(2)
            asym = 0.5 * (mat - np.transpose(mat))

            return tr, sym, asym
        
        def rotation(S: np.array):
            tr, sym, asym = decompose(S)

            h = asym + np.eye(2) * tr / 2.
            s = (np.trace(np.matmul(h, np.transpose(h))) / 2.)**0.5
            rot = h / max(s, 1.e-12)

            return rot, np.arctan2(rot[0, 1], rot[0, 0])


        x = vertices_x - center[0]
        y = vertices_y - center[1]

        if len(x) < 2:
            return np.nan, np.nan, np.nan, np.nan, np.nan

        # sort the vertices
        angles = np.arctan2(y, x)
        order = np.argsort(angles)
        x = x[order]
        y = y[order]

        dual_A = 0
        q = np.zeros((2, 2))
        for i in range(len(x)):
            # vertices of a triangle
            ax = 0
            ay = 0

            bx = x[i]
            by = y[i]

            cx = x[(i + 1) % len(x)]
            cy = y[(i + 1) % len(x)]


            # transform to shape tensor
            base_1 = np.array([[bx - ax, cx - bx],
                               [by - ay, cy - by]])
            base_equilateral = np.array([[1.0, -0.5],
                                        [0.0, 3.**0.5 / 2.]])
            
            # the triangle shape in reference to equilateral triangle
            S = np.matmul(base_1, np.linalg.inv(base_equilateral))

            # decompose shape tensor
            tr_S, sym_S, asym_S = decompose(S)

            # rotational component
            rot, theta = rotation(S)

            # area
            area = np.linalg.det(S)

            # shape deformation
            s = (np.trace(np.matmul(sym_S, np.transpose(sym_S))) / 2.)**0.5

            _q = (  np.arcsinh(s / area**0.5) / max(s, 1.e-12)
                  * np.matmul(sym_S, np.transpose(rot)))
            
            dual_A += area
            q += area * _q

        q /= dual_A

        abs_q = (np.trace(np.matmul(q, np.transpose(q))) / 2.)**0.5
        ratio_WH = np.exp(2 * abs_q)
        theta = np.arctan2(q[1, 0], q[0, 0]) / 2.
        
        return q, dual_A, theta, abs_q, ratio_WH

In [ ]:
q, dual_A, theta, abs_q, ratio_WH = elongation(vertices_x, vertices_y, center)
print(ratio_WH) # compare to ratio when creating vertices

In [ ]:
def rescale_coordinates(coordinates_x, coordinates_y, center, *, ratio_WH, theta):
    """Rescale the vertices from ellipse to circle"""
    displ_x = coordinates_x - center[0]
    displ_y = coordinates_y - center[1]

    dx_ = (displ_x * np.cos(-theta) - displ_y * np.sin(-theta)) / ratio_WH
    dy_ =  displ_x * np.sin(-theta) + displ_y * np.cos(-theta)

    dx = dx_ * np.cos(theta) - dy_ * np.sin(theta)
    dy = dx_ * np.sin(theta) + dy_ * np.cos(theta)

    return dx, dy

In [ ]:
vertices_x_corrected, vertices_y_corrected = rescale_coordinates(
    vertices_x, vertices_y, center,
    ratio_WH=ratio_WH, theta=theta
)

coordinates_x_corrected, coordinates_y_corrected = rescale_coordinates(
    coordinates_x, coordinates_y, center,
    ratio_WH=ratio_WH, theta=theta
)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect(1)
ax.scatter(center[0], center[1], color='black', label='center HC')
ax.scatter(vertices_x, vertices_y, color='grey', label='vertices')
ax.scatter(
    vertices_x_corrected, vertices_y_corrected, 
    color='gray', marker='x',
    label='corrected vertices'
)
ax.quiver(
    vertices_x_corrected,
    vertices_y_corrected, 
    np.roll(vertices_x_corrected, 1) - vertices_x_corrected,
    np.roll(vertices_y_corrected, 1) - vertices_y_corrected,
    angles='xy', scale_units='xy', scale=1,
    headwidth=0, headlength=0,headaxislength=0,
    color='gray',
    label='junctions'
)

ax.scatter(coordinates_x, coordinates_y, color='red', label='HCs')
ax.scatter(
    coordinates_x_corrected, coordinates_y_corrected, 
    color='red', marker='x',
    label='corrected HCs'
)
ax.quiver(
    coordinates_x_corrected,
    coordinates_y_corrected, 
    center[0] - coordinates_x_corrected,
    center[1] - coordinates_y_corrected,
    angles='xy', scale_units='xy', scale=1,
    headwidth=0, headlength=0,headaxislength=0,
    color='red',
)


ax.legend(bbox_to_anchor=(1.05, 1))
ax.set_title("Arrangement of vertices and HCs after correction for elongation")

In [ ]:
print(elongation(vertices_x_corrected, vertices_y_corrected, center)[4])
# This should be 1, as the corrected polygon has similar long and short axis

In [ ]:
angles_corrected = np.arctan2(
    coordinates_y_corrected - center[1],
    coordinates_x_corrected - center[0]
)

# calculate the hexatic parameter corrected for cellular elongation
# for the arrangement of HCs around the center cell
print(hexatic_order_parameter(angles_corrected))